In [248]:
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

# Retrograde Analyse für das Schach-Endspiel

Ein Schachspiel ist gewonnen, wenn die gegnerische Figur mattgesetzt wurde. Ein Matt wird erreicht, wenn der Gegner
nach einem Zug keine Möglichkeit hat, seine Figuren so zu bewegen, dass sein König im nächsten Zug *nicht* geschlagen
wird.

Bei einer geringen Anzahl an Figuren $P$ im Spielzustand lassen sich alle möglichen Positionen und damit Züge berechnen.
Aus dieser Rechnung kann eine Strategie entwickelt werden, den Gegner zu schlagen.

Folgend werden folgende Definitionen verwendet:
* $board.pieces$: Liste der Figuren, welche in einem Zustand vorhanden sind.
* $validBoards$: Alle Zustände des Schachspiels, die gegen keine Regeln verstoßen.
* $wonBoards$: Alle Zustände des Schachspiels, in denen ein Spieler gewonnen hat.
* $previousStates(b)$: Alle Zustände, aus denen durch Ausführen eines einzelnen Zuges der Zustand $b$ erreicht werden kann.

Seien alle möglichen (validen) Kombinationen von Positionen der Figuren $P$ die Menge $S$.  
Für $S$ gilt:  
$$
board \in S \implies \forall p \in P : p \in board.pieces \\
\land \\
board \in S \implies board \in validBoards
$$

Aus der Menge $S$ lassen sich Zustände auswählen, welche $n$ Züge vom Sieg entfernt sind. 
Diese Zustände lassen sich in $S_n$ zusammenfassen. Ist das Spiel gewonnen, verbleiben 0 Züge bis zum Sieg.  
Für alle diese Zustände, in denen ein Spieler mattgesetzt ist, gilt:  
$$board \in S_0 \implies board \in wonBoards \land board \in S$$  
Aus dieser Definition können induktiv die verbleibenden $S_n$ hergeleitet werden:  
$$board \in S_{n+1} \iff board \in S \land \exists b \in S_n: board \in previousStates(b)$$ 

Für die Berechnungen in diesem Notebook gilt da für den schwarzen Spieler immer nur der König auf dem Feld steht weiter Folgendes:
* $board.turn$: Der Spieler, welcher am Zug ist

$$
n \equiv 0 \mod 2 \implies \forall b \in S_n : b.turn = schwarz \\
\land \\
n \equiv 1 \mod 2 \implies \forall b \in S_n : b.turn = weiß \\
$$

Dieses Notebook wird zur Berechnung der $S_n$ Mengen verwendet. Diese werden benötigt, um letztendlich ein Schach-Endspiel lösen zu können. 

## Imports

Im Rahmen dieses Notebooks werden für die Darstellung und Durchführung der besagten Analyse folgende drei Bibliotheken 
verwendet:

* [``chess``](https://python-chess.readthedocs.io/en/latest/): Die Python-Schach-Bibliothek, mit welcher Spielzustände 
dargestellt werden, Züge und Zustände ausgewertet etc.
* [``clear_output``](https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html) aus der 
IPython-display-Library: Eine Funktion, die für die Fortschrittsanzeige von Berechnungen verwendet wird.
* [``pickle``](https://docs.python.org/3/library/pickle.html): Wird zum Speichern der Ergebnisse verwendet.
* [``os``](https://docs.python.org/3/library/os.html): Wird zum Löschen einer temporären Datei verwendet.

In [249]:
import chess
from enum import Enum
from IPython.display import clear_output
import json
import pickle
import os
from zipfile import ZipFile, ZIP_DEFLATED

## Funktionen zur Bestimmung aller gültigen Positionen

Ein Schachbrett besteht aus insgesamt acht Spalten und Zeilen. Die Spalten werden durch Buchstaben gekennzeichnet, 
die Zeilen durch Zahlen. Aus der Kombination einer Spalte (z.B. a) und einer Zahl (z.B. 1) erhält man eine 
eindeutige Kennzeichnung für ein Feld (z.B. a1).  
Die folgende Funktion kombiniert die Buchstaben a bis h mit den Zahlen 1 bis 8 zu Feldnamen und gibt diese zurück.

In [250]:
def get_all_squares():
    columns = {
        1 : 'a',
        2 : 'b',
        3 : 'c',
        4 : 'd',
        5 : 'e',
        6 : 'f',
        7 : 'g',
        8 : 'h'
    }
    
    all_squares = []
    for row in range(1,9):
        for col_num in range(1,9):
            column = columns[col_num]
            all_squares.append(chess.parse_square(column + str(row)))
            
    return all_squares

Das Erstellen jeglicher Boards wird mit der Funktion ``place_piece_everywhere_on_every_board`` umgesetzt. 
Diese erhält folgende Parameter: 

* ``piece``: Die zu platzierende Figur als Objekt der chess-Library
* ``list_of_boards``: Eine Liste mit Board-Objekten, auf welchen die Figur platziert werden soll

Die Funktion betrachtet jede Situation in der `list_of_boards`. Das übergebene `piece` wird auf jeden freien 
Platz dieser Situation platziert. Jedes Mal, wenn eine Figur platziert wird, wird eine Kopie des Board-Objektes erstellt, 
die ``list_of_boards`` wird folglich nicht verändert.  
Wenn der zweite König platziert wird, wird die Situation zusätzlich auf Validität überprüft.
Wenn alle Figuren platziert wurden, werden nur Boards, in denen Schwarz matt ist, zurückgegeben. 


//TODO: Brauchen wir das? Stroetmann sagte ja das ist bei simplen Funktionen nicht nötig
* Über alle Spielbretter in der ``list_of_boards`` iterieren.
* Die bereits verwendeten Spielfelder speichern.
* Über alle Spielfelder iterieren.
* Wenn das Spielfeld noch nicht benutzt ist:
  * Das Spielbrett kopieren und die Figur auf diesem Feld platzieren.
  * Die Kopie der Ergebnisliste anfügen.

Die Funktion gibt als Ergebnis eine Liste aller generierten Zustände gespeichert in ``result_list`` zurück.

In [251]:
def place_figure_everywhere_on_every_board(piece, list_of_boards, piece_count):
    result_list = []
    uniques = []
    all_squares = get_all_squares()
    used_boards = set()
    
    for board in list_of_boards:
        squares_used = list(board.piece_map().keys())
        for square in all_squares:
            if square not in squares_used:
                tmp_board = board.copy()
                tmp_board.set_piece_at(square, piece)
                
                if len(squares_used) >= 1 and not tmp_board.is_valid(): 
                    # Don't process invalid boards further than the second king
                    continue
                    
                outcome = tmp_board.outcome()
                if outcome is not None:
                    if outcome.winner is not None and tmp_board.is_valid() and (tmp_board.turn, tmp_board.__str__()) not in used_boards:
                        uniques.append(tmp_board)
                        result_list.append(tmp_board)
                        used_boards.add((tmp_board.turn,tmp_board.__str__()))
                        for swt in Swap_Type:
                            mir_board = mirror(tmp_board, swt)
                            if (mir_board.turn, mir_board.__str__()) not in used_boards:
                                result_list.append(mir_board)
                                used_boards.add((mir_board.turn,mir_board.__str__()))
                        continue
                if len(squares_used) + 1 < piece_count: #Board is valid, but needs more pieces
                    result_list.append(tmp_board)
    return result_list, used_boards, uniques

## Die Ursprungsmenge $S_0$ erstellen
Als Basis der Berechnung dient die Liste $S_0$. Diese enthält alle möglichen Konstellationen der Spielfiguren auf dem
Spielbrett, in denen Weiß Schwarz besiegt hat.
Hierfür werden die Figuren mit der Funktion ``place_figure_everywhere_on_every_board`` auf allen 
Positionen platziert.   

Die Funktion `setup_boards` übernimmt dies und gibt sowohl die Liste $S_0$ als auch eine Menge an Tupeln der Form
`(Bool, String)` zurück, welche verwendet wird, um effizient bereits verwendete Boards zu identifizieren.

In [252]:
def setup_boards(user_supplied_pieces):
    pieces_to_place = create_piece_list(user_supplied_pieces)
    
    empty_board = chess.Board().empty()
    empty_board.turn = chess.BLACK
    s_0 = [empty_board]
    
    piece_count = len(pieces_to_place)
    for piece in pieces_to_place:
        s_0, used_boards, uniques = place_figure_everywhere_on_every_board(piece, s_0, piece_count)

    print(str(len(s_0)) + " Boards in S_0")
    return s_0, used_boards, uniques

# A queen will automatically be replaced by a pawn
def create_piece_list(user_supplied_pieces):
    if chess.Piece.from_symbol("P") in user_supplied_pieces:
        user_supplied_pieces.remove(chess.Piece.from_symbol("P"))
        user_supplied_pieces.append(chess.Piece.from_symbol("Q"))
        
    return [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k")] + user_supplied_pieces

Der nächste Schritt besteht darin, sämtliche $S_{n}$ Mengen zu bestimmen. Hierzu werden die bereits bestimmten $S_n$
Mengen genommen und alle Bretter aus S selektiert, die innerhalb eines Zugs in $S_n$ landen. Die Umsetzung erfolgt 
durch die Funktion ``previous_states``. Diese erhält unter anderem zwei Listen, mit denen dieses Verfahren durchgeführt
werden soll. Alle Funktionsparameter können aus der nachfolgenden Liste entnommen werden:

* ``s_n``: Die Liste mit Board-Objekten, deren Spielbretter sich $n$ Züge vom Sieg entfernt befinden.
* ``s``: Die Liste, aus welcher die nächste Stufe ($n+1$) ausgewählt werden soll.
* ``s_white_ascii``: Liste der ASCII Repräsentationen aller bereits zugeordneten Spielbretter mit weiß am Zug. Die ASCII Repräsentationen sind ein Zwei-Tupel aus der Farbe am Zug und einer ASCII-Darstellung des Spielbrettes, welche von der Chess-Library generiert wird.
* ``iteration_count``: $m$ des $S_m$, welches gerade berechnet wird.

Der Algorithmus zur Bestimmung der Menge $S_{n+1}$ wird im folgenden Abschnitt beschrieben.

* Für effizientere Vergleiche wird für jedes Objekt in $S$ die ASCII-Repräsentation gespeichert.
* Über die Objekte in $S_n$ iterieren.
  * Den Spieler, welcher am Zug ist, wechseln (Da, um im aktuellen Zustand anzukommen, der andere Spieler einen Zug
  gemacht hat)
  * Pseudo legale Spielzüge von der Bibliothek berechnen lassen:
    Ein pseudo legaler Spielzug ist ein Spielzug, welcher die grundsätzlichen Bewegungsregeln der Figur einhält,
    aber das Schachbrett unter Umständen in einen nicht regelkonformen Zustand versetzt.
  * Über diese Spielzüge iterieren.
    * Den Zug ausführen.
    * Überprüfen, ob das Board sich in einem erlaubten Zustand befindet.
      Wenn nein, den Schleifendurchlauf abbrechen.
    * Mittels der ASCII-Repräsentation überprüfen, ob das modifizierte Spielbrett in $S$ gefunden wird.
    * Wenn es gefunden wird, die ASCII-Repräsentation zu $s_{n+1-ASCII}$ hinzufügen.
    * Den Zug rückgängig machen.
  * Den Spieler, welcher ursprünglich am Zug war, wiederherstellen.
* Über $S$ iterieren.
  * Wenn sich das Spielbrett in $S_{n+1-ASCII}$ befindet, wird es $S_{n+1}$ hinzugefügt.
    * Besonderheit für Spielbretter, bei welchen Schwarz am Zug ist: Diese werden nur zu $S_{n+1}$ hinzugefügt, wenn
    alle Züge, welche von dieser Position möglich sind, in einem $S_m$ mit $m < n$ enden.
    Für diese Überprüfung wird $S_{white-ASCII}$ verwendet.
  * Wenn sich das Spielbrett nicht in $S_{n+1-ASCII}$ befindet, wird es $S_{tmp}$ hinzugefügt.
* Wurden Positionen mit Weiß am Zug berechnet, diese $S_{white-ASCII}$ hinzufügen.

Außerdem müssen bei der Durchführung des Algorithmus weitere Aspekte berücksichtig werden:

* Da Bauern nur in eine Richtung laufen können, müssen die rückwärts Schritte eines Bauern manuell durchgeführt werden. Bauern werden daher im ersten Schritt ignoriert.
* Bauern, die die oberste Reihe des Spielfeldes erreichen, können zu einer anderen Figur eingetauscht werden. Dieser Schritt wird nicht durch die Pseudo-Legal-Moves abgedeckt, daher wird, sollte sich eine Königin in der obersten Reihe befinden, diese manuell durch einen Bauern ersetzt.

Sobald die Funktion ausgeführt wurde, wird ein Tupel bestehend aus den Mengen $S_{n+1}$, $S_{tmp}$ und $S_{white-ASCII}$ zurückgegeben.

In [286]:
def previous_states(s_n, used_boards, iteration_count, user_wants_pawn, uniques):
    #variables
    s_n1 = []
    s_n1_tuples = set()
    s_n1_uniques = []

    for i in range(len(uniques)):
        status = "Calculating S" + str(iteration_count) + " - Board " + str(i+1) + " of " + str(len(s_n)) + " from S" + str(iteration_count-1)
        # clear_output(wait=True)
        print(status)
        print("Recent Uniques size: " + str(len(s_n1_uniques)))
        print("Recent S size: " + str(len(s_n1)))

        # Copy current board and invert the player
        chess_board = uniques[i].copy()
        chess_board.turn = chess_board.turn ^ True

        # Find all Pawns
        pawn_positions = find_pawns(chess_board)
        
        # try moves and check if they lead to new boards
        tmp_list, tmp_set, tmp_uniques = regular_moves(chess_board, used_boards, s_n1_tuples, pawn_positions)
        s_n1 += tmp_list
        s_n1_tuples |= tmp_set
        s_n1_uniques += tmp_uniques

        if user_wants_pawn:
            # Push all pawns one row back and check if this leads to new boards
            if len(pawn_positions) > 0:
                tmp_list, tmp_set = pawn_moves(chess_board, used_boards, s_n1_tuples)
                s_n1 += tmp_list
                s_n1_tuples |= tmp_set
            
            # Exchange Queens with Pawns
            queen_positions = check_top_row_for_queen(chess_board)
            if queen_positions:
                tmp_list, tmp_set = replace_queen_with_pawn(chess_board, used_boards, s_n1_tuples, queen_positions)
                s_n1 += tmp_list
                s_n1_tuples |= tmp_set
        else:
            pass
            '''
            for swt in Swap_Type:
                mirrored_board = mirror(chess_board,swt)
                if mirrored_board.is_valid(): 
                    tmp_list, tmp_set = regular_moves(mirrored_board, used_boards, s_n1_tuples, pawn_positions)
                    s_n1 += tmp_list
                    s_n1_tuples |= tmp_set

            '''
            

        # Restore the original state of the board
        chess_board.turn = chess_board.turn ^ True

    # Only needed for Black-Moves
    if iteration_count % 2 == 0:
        # clear_output(wait=True)
        print("Calculating S" + str(iteration_count) + " - Checking Black Moves for determinism")
        print("Before checking: " + str(len(s_n1)))
        s_n1, s_n1_tuples = check_black_determinism(s_n1, used_boards)
        print("After checking: " + str(len(s_n1)))

    # clear_output(wait=True)
    print("Done with S" + str(iteration_count))
    print("S_N_1_Tuples: " + str(len(s_n1_tuples)))
    print(s_n1_tuples)
    return s_n1, used_boards | s_n1_tuples, s_n1_uniques

## Hilfsfunktionen für die Berechnung

Die folgenden Funktionen werden zur Berechnung der previous_states verwendet. 
Sie übernehmen dabei diverse Aufgaben wie das Durchführen von regulären Moves oder das "manuelle" Versetzen von Figuren, um eine andere Situation zu generieren. 


Die Funktion ``regular_moves`` führt für eine übergebene Situation `chess_board` alle `pseudo_legal_moves` durch, um mögliche vorhergehende Situationen zu berechnen.  
Pseudo-Legale-Züge sind Züge, welche die Figuren auf eine Art bewegen, die der Figur gestattet ist, aber unter Umständen in eine nicht legale Spielsituation führt. 
Diese werden verwendet, da nur weil der Move von $S_{n+1}$ zu $S_n$ legal ist, der Zug umgekehrt dies nicht sein muss.
Ein simples Beispiel:
Situation $S_n$: Ein König befindet sich ein Feld von einem Schach entfernt.  
Diese Position kann erreicht worden sein, da der König von einer Position in $S_{n+1}$ sich aus diesem Schach herausbewegt hat.
Der Zug "in das Schach", wäre jedoch nicht legal, weshalb ein Move aus der Liste der `pseudo_legal_moves` zur Berechnung genommen werden muss.
Dies funktioniert nicht für Bauern, da ein Schritt nach "hinten" keine Bewegung ist, welche der Figur zusteht.

Die Funktion überprüft jeden Zug, welcher in der Situation möglich ist. Wenn die errechnete Situation valide und noch nicht verwendet (überprüft durch Einträge in `used_boards` und `s_n1_tuples`) ist, 
wird sie den Rückgabe-Variablen angefügt.

In [272]:
def regular_moves(chess_board, used_boards, s_n1_tuples, pawn_positions):
    new_boards = []
    new_tuples = set()
    new_uniques = []
    for pLMove in chess_board.pseudo_legal_moves:
        if chess.square_name(pLMove.from_square) not in pawn_positions:
            chess_board.push(pLMove)
            chess_board.turn = chess_board.turn ^ True
            if not chess_board.is_valid():
                chess_board.turn = chess_board.turn ^ True
                chess_board.pop()
                continue
            # If the new board is found in S, it can be reached in one step
            tuple_rep = (chess_board.turn,chess_board.__str__())
            if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples and tuple_rep not in new_tuples:
                print("Uniques:" + str(len(new_uniques)))
                new_uniques.append(chess_board.copy())
                print("Boards: " + str(len(new_boards)))
                new_boards.append(chess_board.copy())
                new_tuples.add(tuple_rep)
                for swtype in Swap_Type:
                    mirrored_board = mirror(chess_board, swtype)
                    tuple_rep_mir = (mirrored_board.turn,mirrored_board.__str__())
                    if tuple_rep_mir not in used_boards and tuple_rep_mir not in s_n1_tuples and tuple_rep_mir not in new_tuples:
                        new_boards.append(mirrored_board.copy())
                        new_tuples.add(tuple_rep_mir)
            chess_board.turn = chess_board.turn ^ True
            chess_board.pop()
    
    return new_boards, new_tuples, new_uniques

Die quadratische Natur des Schachbrettes ermöglicht es das Spielbrett zu spiegeln / rotieren und weitere Situationen zu erhalten. 
Die Formeln zum Rotieren der Spielsituationen wurden [dieser Quelle](https://www.chessprogramming.org/Flipping_Mirroring_and_Rotating) entnommen.

In [255]:
class Swap_Type(Enum):
    VERTICAL = "vertical"
    HORIZONTAL = "horizontal"
    #DIAGONAL_FROM_BOTTOM = "diagonal_from_bottom"
    #DIAGONAL_FROM_TOP = "diagonal_from_top"
    #ROTATE = "rotate_right"

def mirror(board, sw_type : Swap_Type):
    swaps = {
        "vertical" : {x:x^56 for x in range(64)},
        "horizontal" : {x:x^7 for x in range(64)},
        "diagonal_from_bottom" : {x:(((x >> 3) | (x << 3)) & 63) for x in range(64)},
        "diagonal_from_top" : {x:(((x >> 3) | (x << 3)) & 63) ^ 63 for x in range(64)},
        "rotate_right" : {x:(((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)}
    }
    
    swapped_board = chess.Board()
    swapped_board.clear()
    swapped_board.turn = board.turn

    for position, piece in board.piece_map().items():
        swapped_board.set_piece_at(swaps[sw_type.value][position], piece)
    
    return swapped_board

Befinden sich Bauern in der Situation, müssen diese manuell platziert werden, da für diese auch in den `pseudo_legal_moves` nur die Züge $xn \rightarrow x(n+1)$ aufgeführt sind.
Die Funktion `pawn_moves` erfüllt diese Anforderung.
Ähnlich wie die Funktion `regular_moves` werden für eine Situation `chess_board` alle Situationen berechnet, welche durch Bewegung eines Bauerns zu `chess_board` werden.  
Auch diese Situationen werden sowohl auf Validität als auch bisheriges Vorkommen überprüft, bevor sie den Rückgabevariablen angefügt werden.

In [256]:
def pawn_moves(chess_board, used_boards, s_n1_tuples):
    new_boards = []
    new_tuples = set()
    
    for pawn in chess_board.pieces(chess.PAWN, True):
        if chess_board.piece_at(pawn - 9) is None:
            chess_board.remove_piece_at(pawn)
            chess_board.set_piece_at(pawn - 9, chess.Piece.from_symbol('P'))
            if chess_board.is_valid():
                tuple_rep = (chess_board.turn,chess_board.__str__())
                if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples:
                    new_boards.append(chess_board.copy())
                    new_tuples.add(tuple_rep)
            chess_board.remove_piece_at(pawn - 9)
            chess_board.set_piece_at(pawn, chess.Piece.from_symbol('P'))
            
    return new_boards, new_tuples


Ein Problem, das bei der Verwendung der Rückwärts-Analyse auftritt, liegt in dem Szenario: 
"König und Bauer gegen König". Dieses Szenario beinhaltet die Umwandlung des Bauerns, welcher die oberste Zeile erreicht hat, in eine andere Figur (Dame, Turm, Läufer, Springer). 
Da die Dame die stärkste Figur im Spiel ist, wird immer dieser Tausch gewählt.
Hat der Nutzer bei den weißen Figuren, welche sich in der Situation sollen, einen Bauern angegeben, wurde dieser beim Errechnen der Menge $S_0$ durch eine Königin ersetzt.

Für die Berechnung der idealen Züge muss der Bauer wieder in die Situationen, welche sich in den $S_n$ Mengen befinden, eingeführt werden.
Der Tausch eines Bauerns zu einer Dame kann nicht durch die `pseudo_legal_moves` umgekehrt werden.

Die Funktion ``check_top_row_for_queen`` überprüft, ob ein solcher Tausch möglich ist. Sie erhält als Parameter eine Situation `board`, für welches die Felder der obersten Zeile überprüft und jedes zurückgegeben wird, auf dem sich eine Dame befindet.

In [257]:
def check_top_row_for_queen(board):
    return_list = []
    for i in range(56, 64):
        if board.piece_type_at(i) == chess.QUEEN:
            return_list.append(i)

    if len(return_list) > 0:
        return return_list
    else:
        return False

Wurden mittels der vorhergehenden Funktion Damen in der obersten Zeile gefunden, ersetzt `replace_queen_with_pawn` alle diese Positionen (`toprow_queen_positions`) durch einen 
Bauern in der vorletzten Zeile.

In [258]:
def replace_queen_with_pawn(chess_board, used_boards, s_n1_tuples, toprow_queen_positions):
    new_boards = []
    new_tuples = set()
    
    for square in toprow_queen_positions:
        chess_board.remove_piece_at(square)
        chess_board.set_piece_at(square - 9, chess.Piece.from_symbol('P'))
        if chess_board.is_valid():
            tuple_rep = (chess_board.turn,chess_board.__str__())
            if tuple_rep not in used_boards and tuple_rep not in s_n1_tuples:
                new_boards.append(chess_board.copy())
                new_tuples.add(tuple_rep)
        chess_board.remove_piece_at(square - 9)
        chess_board.set_piece_at(square, chess.Piece.from_symbol('Q'))
    return new_boards, new_tuples

Da Bauern mittels der Funktion `pawn_moves` gesondert behandelt werden müssen, muss in `regular_moves` verhindert werden, dass Züge mit Bauern durchgeführt werden.
Hierfür wird die Information benötigt, auf welchen Feldern sich ein Bauer befindet. Diese Information wird durch die Funktion `find_pawns` generiert.  

In [259]:
def find_pawns(chess_board):
    result = []
    for pawn in chess_board.pieces(chess.PAWN, True):
            result.append(chess.square_name(pawn))
    return result

Wenn mittels der Ki eine Spielsituation ausgewertet wird, kann für jeden Zug des weißen Spielers ein Zug ausgewählt werden.
Für die Situationen, bei denen Schwarz am Zug ist, muss die Ki alle möglichen Züge auswerten können.
Da jedoch für einen spezifischen Zug, welcher eine Situation von $S_n$ in $S_{n-1}$ führt, dasselbe nicht für alle Züge, welche in der Situation möglich sind gilt,
müssen die Situationen, bei welchen Schwarz am Zug ist, besonders gefiltert werden.
Für jede Situation $b$ aus einem $S_n$ mit $n \% 2 \equals 0$ muss folglich gelten:  
$$
b \in S_n \implies \forall m \in valid\_moves(b): b.push(m) \in S_{m} \land m < n
$$
Wobei `valid_moves` die Liste der legalen Züge für eine Situation ist und `b.push(m)` die Situation beschreibt, welche durch Ausführen des Zuges $m$ entsteht. 

In [260]:
def check_black_determinism(s_n1, used_boards):
    s_n1_tmp = []
    s_n1_tuples_tmp = set()
    
    for chess_board in s_n1:
        include = True
        if not chess_board.turn:
            for move in chess_board.legal_moves:
                chess_board.push(move)
                tuple_rep = (chess_board.turn,chess_board.__str__())
                chess_board.pop()
                if tuple_rep not in used_boards:
                    include = False
        if include:
            s_n1_tmp.append(chess_board)
            s_n1_tuples_tmp.add((chess_board.turn,chess_board.__str__()))
    
    return s_n1_tmp, s_n1_tuples_tmp

## Export in Datei
Nach erfolgreicher Berechnung eines $n$ werden die FENs der Boards in eine Datei geschrieben.
Nach erfolgreicher Berechnung aller $S_n$ wird die temporäry Datei in eine `.chessAI` Datei für die Verwendung in der Ki und eine `.chessTest` für das 
Testen der Ergebnisse konvertiert.

Damit keine Werte einer vergangenen Berechnung in der temporären Datei vorliegen, muss zuerst eine leere `.preConvert` Datei erstellt werden. 

In [261]:
def create_empty_file(filename):
    f = open("S_n_Results/" + filename + ".preConvert", "w")
    f.write("")
    f.close()

Die Zwischenergebnisse werden stetig mittels der `append_to_file` Funktion an die zuvor erstellte `.preConvert` Datei angehängt.
Jede Zeile entspricht hierbei einem $n$ aus den $S_n$ Mengen.
Für die Zwischenergebnisse werden die FENs als JSON gespeichert.

In [262]:
def append_to_file(s_n, filename):
    s_n_ascii = []
    for board in s_n:
        s_n_ascii.append(board.fen())

    f = open("S_n_Results/" + filename + ".preConvert", "a")
    f.write("\n")
    f.write(json.dumps(s_n_ascii))
    f.close()

Nachdem die gesamte Sequenz aller $n$ berechnet wurde, muss die temporäre Datei in zwei Dateien zur Auswertung konvertiert werden.
Die `.chessAI` Datei enthält Listen von Tupeln, bestehend aus dem Spieler, welcher am Zug ist (repräsentiert durch einen `boolean`) und der ASCII-Repräsentation des Boards.
Die `.chessTest` Datei enthält die FENs, um in den Test-Szenarien wieder Board-Objekte erstellen zu können.

Zum Speichern der Dateien wird aus Effizienzgründen das Modul Pickle verwendet, welches die Daten in Binärdateien speichert.  

In [263]:
def convert_file(filename):
    s_n_seq_fens = []
    s_n_seq_tuples = []
    f = open("S_n_Results/" + filename + ".preConvert", "r")
    lines = f.readlines()
    first = True
    for line in lines:
        # First line is empty
        if first:
            first = False
            continue

        tmp_list = []
        tmp_set = set()
        tmp = json.loads(line)

        for fen in tmp:
            tmp_list.append(fen)
            tmp_board = chess.Board(fen)
            tmp_set.add((tmp_board.turn,tmp_board.__str__()))
        s_n_seq_fens.append(tmp_list)
        s_n_seq_tuples.append(tmp_set)
    f.close()

    f = open("S_n_Results/" + filename + ".pickle", "wb")
    f.write(pickle.dumps(s_n_seq_tuples))
    f.close()
    
    f = open("S_n_Results/" + filename + ".chessTest", "wb")
    f.write(pickle.dumps(s_n_seq_fens))
    f.close()
    
    with ZipFile("S_n_Results/" + filename + '.chessAI', 'w', compression=ZIP_DEFLATED) as zipped:
        zipped.write("S_n_Results/" + filename + ".pickle", filename+".pickle")
    if os.path.exists("S_n_Results/" + filename + ".chessAI") and os.path.exists("S_n_Results/" + filename + ".pickle"):
        os.remove("S_n_Results/" + filename + ".pickle") 

    if os.path.exists("S_n_Results/" + filename + ".preConvert"):
        os.remove("S_n_Results/" + filename + ".preConvert")

Mit den zuvor definierten Funktionen ist es nun möglich, alle $S_n$ zu bestimmen. Hierfür wird die Funktion ``next_states`` solange aufgerufen, bis keine Situationen
für ein $n+1$ mehr gefunden werden und die Liste $s_n$ leer ist.

Die Funktion `run` geht hierbei von einem `s_n`, welches zuvor bestimmt wurde, aus und durchläuft diesen Prozess in einer Schleife.
Nach abgeschlossener Rechnung wird das Konvertieren der Datei gestartet.

In [284]:
def run(s_n, used_boards, n, user_wants_pawn, filename, uniques):
    if n == 0:
        append_to_file(s_n, filename)

    for i in range(2):
        n += 1
        s_n, used_boards, uniques = previous_states(s_n, used_boards, n, user_wants_pawn, uniques)
        print("S " + str(n) + ": " + str(len(s_n)))
        if not s_n: #an empty list is false
           break
        append_to_file(s_n, filename)

    print("Done")
    print(str(n) + " S-Lists calculated")
    print(uniques)

    # convert_file(filename)

    print("File converted")

Die Funktion `run_from_start` berechnet $S_0$ und startet anschließend die Berechnung aller $S_n$

In [265]:
def run_from_start(user_supplied_pieces, filename):
    user_wants_pawn = chess.Piece.from_symbol("P") in user_supplied_pieces
    
    s_0, used_boards, uniques = setup_boards(user_supplied_pieces)
    print(len(used_boards))
    print(len(uniques))
    create_empty_file(filename)
    run(s_0, used_boards, 0, user_wants_pawn, filename, uniques)

Wurde die Berechnung abgebrochen (z.B. durch einen Neustart des Computers) kann die Funktion `resume_from_file` den Zustand, welcher in einer `.preConvert` Datei gespeichert wurde
wiederherstellen, und die Berechnung beim nächsten $n$ fortsetzen.
Hierfür werden alle bereits berechneten $n$ in Objekte instanziiert und die Tupel Repräsentationen als bereits verwendete Situationen (`used_boards`) gespeichert.
Für das höchste $n$, werden die Objekte behalten und der `run` Funktion übergeben.

In [266]:
def resume_from_file(filename):
    used_boards = set()
    s_n = []
    count = -1

    f = open("S_n_Results/" + filename + ".preConvert", "r")
    lines = f.readlines()
    first = True
    for line in lines:
        # First line is empty
        if first:
            first = False
            continue

        tmp_list = []
        tmp_set = set()
        tmp = json.loads(line)

        for fen in tmp:
            chess_board = chess.Board(fen)
            tmp_list.append(chess_board)
            tmp_set.add((chess_board.turn,chess_board.__str__()))
        s_n = tmp_list
        used_boards |= tmp_set
        count += 1
    f.close()
    print("Starting at S" + str(count) + " (" + str(len(s_n)) + " Boards)")
    
    # Adding a few Pawns to the result only slightly increases the file_size, and the value of user_wants_pawn is \
    # currently not stored
    run(s_n, used_boards, count, True, filename)

## Konfigurations Variablen

Für die Bestimmung der gewonnenen Spielbretter müssen nun die Spielfiguren angegeben werden, für die die Endspiel-Datenbank berechnet werden soll. Hierfür werden in der Liste ``pieces_to_place`` alle Figuren aufgeführt.
Die Einträge der Liste werden als Tupel bestehend aus Figur und Farbe gespeichert.
Bsp.: ``(chess.KING, chess.WHITE)``
Die Reihenfolge oder Position der Figur ist für die Berechnung irrelevant.
Diese wird erst im nächsten Schritt (der Auswertung) benötigt.

Der ``Filename`` wird für das Speichern der Ergebnisse verwendet.

In [267]:
WHITE_PIECES_TO_PLACE = [chess.Piece.from_symbol('R')]

FILENAME = "S_n_seq_rook_mirror_1_03"

## Start der Rechnung

In [287]:
%%time
run_from_start(WHITE_PIECES_TO_PLACE, FILENAME)

216 Boards in S_0
216
108
Calculating S1 - Board 1 of 216 from S0
Recent Uniques size: 0
Recent S size: 0
Uniques:0
Boards: 0
Uniques:1
Boards: 3
Uniques:2
Boards: 6
Uniques:3
Boards: 9
Uniques:4
Boards: 12
Uniques:5
Boards: 15
Uniques:6
Boards: 18
Calculating S1 - Board 2 of 216 from S0
Recent Uniques size: 7
Recent S size: 21
Uniques:0
Boards: 0
Uniques:1
Boards: 3
Uniques:2
Boards: 6
Uniques:3
Boards: 9
Uniques:4
Boards: 12
Uniques:5
Boards: 15
Uniques:6
Boards: 18
Calculating S1 - Board 3 of 216 from S0
Recent Uniques size: 14
Recent S size: 42
Uniques:0
Boards: 0
Uniques:1
Boards: 3
Uniques:2
Boards: 6
Uniques:3
Boards: 9
Uniques:4
Boards: 12
Uniques:5
Boards: 15
Uniques:6
Boards: 18
Calculating S1 - Board 4 of 216 from S0
Recent Uniques size: 21
Recent S size: 63
Uniques:0
Boards: 0
Uniques:1
Boards: 3
Uniques:2
Boards: 6
Uniques:3
Boards: 9
Uniques:4
Boards: 12
Uniques:5
Boards: 15
Uniques:6
Boards: 18
Calculating S1 - Board 5 of 216 from S0
Recent Uniques size: 28
Recent S size

In [269]:
#resume_from_file(FILENAME)
